# Project 3
Fraud detection. Using dataset from Kaggle.

## Importing Modules

In [57]:
from bs4 import BeautifulSoup
from random import randint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
import time
import pickle
import os
import datetime
import seaborn as sns
import math
import scipy.stats as stats

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, classification_report

import pymysql.cursors

In [58]:
#Constants
RANDOM_STATE = 999
TEST_SIZE = 0.1

#set this to 1 if you want to see this
#notebook create the table and import the CSV into the table
#on the My SQL server
RUN_TABLE_CREATE_IMPORT = 0

## Data Download and Load Into SQL

Note if you want a copy of the data you can download it from [here](https://www.kaggle.com/ntnu-testimon/paysim1) 

In [59]:
#just reading enough to get the names of the columns
data_df = pd.read_csv("../Data/PS_20174392719_1491204439457_log.csv",nrows=5)

In [60]:
#Getting column names
COL_NAMES = data_df.columns
for a_col in COL_NAMES:
    print(a_col)

step
type
amount
nameOrig
oldbalanceOrg
newbalanceOrig
nameDest
oldbalanceDest
newbalanceDest
isFraud
isFlaggedFraud


In [62]:
#What type of data is in each column?
data_df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [63]:
#Establishing connection to mariaDB server
#Note, sever_details.txt is not provided you'll have to ask me for it
server_details_file_name = "/home/magreen/Dropbox/PERSONAL/Documents/Word/server_details.txt"
SERVER_DETAILS = open(server_details_file_name,"r")
line_of_text = SERVER_DETAILS.readline()

host_name, username, mypassword, db_name = line_of_text.split(',')

host_name = host_name.rstrip()
host_name = host_name.lstrip()
username = username.rstrip()
username = username.lstrip()
mypassword = mypassword.rstrip()
mypassword = mypassword.lstrip()
db_name = db_name.rstrip()
db_name = db_name.lstrip()


In [64]:
connection = pymysql.connect(host=host_name, user=username, password=mypassword, db=db_name, cursorclass=pymysql.cursors.DictCursor)

In [65]:
#get a cursor
mycursor = connection.cursor()

In [66]:
#Create table import entire CSV
if RUN_TABLE_CREATE_IMPORT != 0:
    drop_table_query = "drop table if exists paysim_data;"

    mycursor.execute(drop_table_query)
    mycursor.execute("commit;")

    create_table_query = """


create table paysim_data (
    step INT,
    type TEXT,
    amount REAL,
    nameOrig TEXT,
    oldbalanceOrg REAL,
    newbalanceOrig REAL,
    nameDest TEXT,
    oldbalanceDest REAL,
    newbalanceDest REAL,
    isFraud INT,
    isFlaggedFraud INT
    );


    """
    mycursor.execute(create_table_query)
    mycursor.execute("commit;")

    import_query = """


load data infile '/var/lib/mysql-files/data/PS_20174392719_1491204439457_log.csv'
into table paysim_data fields terminated by ','  lines terminated by '\n'
ignore 1 rows;

    """

    mycursor.execute(import_query)
    mycursor.execute("commit;")
    print("table creation and import done")
else:
    print("Skipping MySQL table creation and CSV import")

Skipping MySQL table creation and CSV import


In [ ]:
########################################################################
#
# Search each column in the MySQL table for null. If null remove the row
#
########################################################################
for column_name in COL_NAMES:
    temp_query = """
    
    select * from paysim_data where {} is null;
    
    """.format(column_name)
    
    mycursor.execute("begin;")
    query_result = mycursor.execute(temp_query)
    print("For column = {} here are the null rows:\n".format(column_name))
    print(query_result)
    print("\n")
    
    if query_result != 0:
        row_contents = mycursor.fetchall()
        print("This row contains NULL information:\n")
        
        for bad_row in row_contents:
            print(row_contents)
            
        temp_efface_row_query = """
        delete from paysim_data where {} is null;
        """.format(column_name)
        
        query_result = mycursor.execute(temp_efface_row_query)
        
        #print("result = {}".format(query_result))
        mycursor.execute("commit;")
        #print("result = {}".format(query_result))
        if query_result:
            print("\nsuccessfully removed row\n")
        else:
            print("error: failed to remove row!")
        
    

        
print("\ncheck done.\n")
#Create table

#import CSV Into SQL Table


## Data Cleaning

In [ ]:
data_df.info()

In [ ]:
data_df.describe()

In [ ]:
data_df.shape

In [ ]:
data_df.dropna(inplace=True)

In [ ]:
data_df.shape

In [ ]:
data_df.head()

Imbalanced?

In [ ]:
data_df["isFraud"].value_counts()

**Yes**

## EDA

Initial thought is that `isFraud` is basically a function of `type` and `amount`

one-hot-encoding `type`

In [ ]:
ohe = OneHotEncoder(sparse=False)

np_data_type = np.array(data_df["type"])
np_data_type = np_data_type.reshape(-1,1)
np_data_type.shape



In [ ]:
np_data_type_ohe = ohe.fit_transform(np_data_type)
np_data_type_ohe.shape

In [ ]:
np_data_type_ohe_df= pd.DataFrame(np_data_type_ohe,
                           columns=ohe.get_feature_names(['type']), #create meaningful column names
                           index=data_df.index) #keep the same index values

#np_data_type_ohe_df.head()

#combine continuous and categorical data

data_df = pd.concat([data_df, np_data_type_ohe_df], axis=1)
data_df.head()

A plot to help me understand how mixed together `isFraud==0` is with `isFraud==1` as a function of `type` and `amount`

In [ ]:
NUM_OF_RANDOM_SAMPLES = 6000

plot_data_df = data_df[["step","isFraud","amount","type_CASH_IN","type_CASH_OUT","type_DEBIT","type_PAYMENT","type_TRANSFER"]]
#limit to the first time step
plot_data_df = plot_data_df[ plot_data_df["step"] == 1 ]
#np_plot_data_df_indices = list(np.random.permutation(np.array(list(plot_data_df.index)))[:5000])
#sns.pairplot(plot_data_df.iloc[[np_plot_data_df_indices]], hue='isFraud');

In [ ]:
#This code is generating the index numbers for data frame plot_data_df
#then randomizing the list containing those indices
#then picking the first NUM_OF_RANDOM_SAMPLES from that list
#this list of randomly picked indices will be used to pick the corresponding
#rows out of plot_data_df to be used in the pair plot
#to plot all 6M rows is not practical (on this machine) 
plot_data_df_indices = list(plot_data_df.index)
plot_data_df_indices = np.random.permutation(plot_data_df_indices)
plot_data_df_indices = plot_data_df_indices[:NUM_OF_RANDOM_SAMPLES]
plot_data_df_indices = list(plot_data_df_indices)


In [ ]:
 plot_data_df = plot_data_df.iloc[plot_data_df_indices]

In [ ]:
plot_data_df.shape

In [ ]:
plot_data_df.head()

In [ ]:
sns.pairplot(plot_data_df, hue='isFraud');

Not looking like good parameters :(

## Baseline Model

Random Guessing (50% chance of `isFraud==1` and 50% chance that `isFraud==0` will be the baseline model

In [ ]:
data_df.shape

In [ ]:
X = data_df[["step","amount","type_CASH_IN","type_CASH_OUT","type_DEBIT","type_PAYMENT","type_TRANSFER"]]
Y = data_df["isFraud"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=TEST_SIZE, random_state=RANDOM_STATE)

number_of_test_samples = X_test.shape[0]
y_pred_zero = np.zeros(number_of_test_samples // 2)
y_pred_one = np.ones( number_of_test_samples -  y_pred_zero.shape[0] )

y_pred = np.array([y_pred_zero, y_pred_one])
y_pred = y_pred.reshape(-1,)
y_pred = np.random.permutation(y_pred)




Scoring the baseline model

In [ ]:
target_names = ["NotFraud","Fraud"]
print(classification_report(Y_test, y_pred,target_names=target_names))

Thoughts about random guessing model:

Precision (TP/(TP+FP)) and recall (TP/(TP+FN)) are high for the `NotFraud` class because most (99.97%) of the response variables in the test set are `NotFraud`. Precision and recall are very low for `Fraud` because there are very few chances to guess right and the random guessing model does not perform well in identifying them.

Accuracy is meaningless for the same reason: It's easy to get 50% right as the vast majority of the results belong to one class. 

TODO: generate confusion matrix which will help illustrate this.

## Model Selection

Just some notes about what model to select:

**Logistic Models**
For small datasets, ‘liblinear’ is a good choice, whereas ‘sag’ and ‘saga’ are faster for large ones.

Note that ‘sag’ and ‘saga’ fast convergence is only guaranteed on features with approximately the same scale. You can preprocess the data with a scaler from sklearn.preprocessing.



## Model Evaluation

In [ ]:
#Since I'm using values that are either 1 or 0 along with amount which is a very large number, will normalize the data
#to aid gradient descent
normalizer = StandardScaler()
X_train_normal = normalizer.fit_transform(X_train.values)
X_test_normal = normalizer.transform(X_test.values)

#logisticRModel = LogisticRegressionCV(Cs=100,cv=5,solver="sag",n_jobs=-1,random_state=RANDOM_STATE)
logisticRModel = LogisticRegression(C=100,solver="sag",n_jobs=-1,random_state=RANDOM_STATE)

logisticRModel.fit(X_train_normal,Y_train)

In [ ]:
Y_pred = logisticRModel.predict(X_test_normal)

In [ ]:
print(classification_report(Y_test, Y_pred,target_names=target_names))

In [ ]:
logisticRModel.score(X_test_normal,Y_test)

In [ ]:
Y_test.value_counts()

In [ ]:
Y_pred_df = pd.DataFrame(Y_pred)
Y_pred_df.value_counts()

In [ ]:
logisticRModel.coef_

Thoughts about baseline logistic model:

Precision (TP/(TP+FP)) and recall (TP/(TP+FN)) are high for the `NotFraud` because most(99.97%) of the response variables in the test set are `NotFraud`. Precision and recall are very low for `Fraud` because there are very few chances to guess right and the random guessing model does not perform well in identify them.

Accuracy is meaningless for the same reason: It's easy to get 50% right as the vast majority of the results belong to one class. 

TODO: generate confusion matrix which will help illustrate this.